In [ ]:
import numpy as np

### Open Serial Port
You might need to change 'COM'

In [ ]:
import serial as ser
s = ser.Serial ('COM6', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

### Define Rotation / Transformation Functions

In [ ]:
def Rx (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [1, 0,  0, 0],
        [0, c, -s, 0],
        [0, s,  c, 0],
        [0, 0,  0, 1]
    ])

def Ry (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, 0,  s, 0],
        [0, 1,  0, 0],
        [-s, 0, c, 0],
        [0, 0,  0, 1]
    ])

def Rz (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, -s,  0, 0],
        [s, c,  0, 0],
        [0, 0, 1, 0],
        [0, 0,  0, 1]
    ])

def Tx (x):
    return np.array ([
        [1, 0, 0, x],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Ty (y):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, y],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Tz (z):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

In [ ]:
"""def move(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 93, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{-a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    if res[2,3] < 0:
       raise Exception(f'Illegal z value: {res[2,3]} lower than 0')
    else:
        s.write(command.encode())
        return res
"""   


In [ ]:
"""def move_nocheck(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 87, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

    s.write(command.encode())
    return res
""" 


In [ ]:
#1. Rot z Base 
#2. Tz 71.5mm
#3  Rot y Shoulder
#4  Tz 125mm
#5  Rot y Elbow
#6  Tz 125mm
#7  Rot y Wrist
#8  Tz 60mm
#9 (Wrist Rot)
#10 Tz 132mm
#11 Tx OFFSET 12.5mm for new start position

#def transformation(a):
#    return Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

In [ ]:
#Constraint: 3 degrees of freedom. Angles over 3 are overwritten with 0°.
def fk(a):
    res = Tx(-125) @ Rz(-a[0,0]) @ Tz(71.5) @ Ry(a[1,0]) @ Tz(125) @ Ry(a[2,0]) @ Tz(125) @ Ry(0) @ Tz(60) @ Tz(132)
    return res

In [ ]:
def diff(a):
    pos = fk(a)
    target = np.array ([[270.0],[-60],[155.0]])
    return pos[0:3,3:4] - target

In [ ]:
def move(a):
    #offset for braccio startpoint (hardware specific, use your default robot position angles)
    startpoint = np.array([[90], [90], [87], [90], [90], [65]])
    a_calc = np.copy (startpoint)
    a_calc [0:3,0:1] += a
    command = f'{-a_calc[0,0]} b {a_calc[1,0]} s {a_calc[2,0]} e {a_calc[3,0]} v {a_calc[4,0]} w {a_calc[5,0]} g\r\n'
    
    #apply forward kinematics
    res = fk(a)
    print (res)
    if res[2,3] < 0:
       raise Exception(f'Illegal z value: {res[2,3]} lower than 0')
    else:
        s.write(command.encode())

## Reverse Kinematics

In [ ]:
def j(f,x0):
    h=1E-8
    y0 = f(x0)
    
    col1 = np.copy(x0.astype(float))
    col1[0,0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0.astype(float))
    col2[1,0] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0.astype(float))
    col3[2,0] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    return np.concatenate((col1_df,col2_df,col3_df),axis=1)

#Iterative newton-raphson approximation for angle correction
#Input: phi1,phi2,phi3 (Winkel!)
def approximate(f,x0):
    x = np.copy(x0.astype(float))
    print(f'Jacobian Matrix:\n{j(f,x)}')
    for i in range(5):
        #x -= np.linalg.solve(j(x.reshape(3,)),f(x.reshape(3,)))
        x -= np.linalg.inv(j(f,x)) @ f(x)
        print(f'n={i+1}\n{x}\n')
    return x

In [ ]:
#Test forward kinematics and diff function
B=10 #base
S=45 #shoulder
E=45 #elbow

#Angles -> Coordinates (Forward Kinematics)

a = np.array([[B],[S],[E]])
print (fk(a))
print (diff(a))

In [ ]:
#Coordinates -> Angles (Inverse Kinematics)

anew = approximate(diff,a)
print (fk(anew))

In [ ]:
def rad(phi):
    return phi * np.pi / 180
def c(phi):
    return np.cos(phi)
def s(phi):
    return np.sin(phi)

def dh_calc(a,alpha,d,theta):
    return np.array([
        [c(theta), -s(theta), 0, a],
        [s(theta)*c(alpha), c(theta)*c(alpha), -s(alpha), -d * s(alpha)],
        [s(theta)*s(alpha), c(theta)*s(alpha), c(alpha), d * c(alpha)],
        [0,0,0,1]
    ])

In [ ]:
b_a = rad(-90)
r1 = np.array([
    [c(b_a),-s(b_a),0,0],
    [s(b_a),c(b_a),0,0],
    [0,0,1,0],
    [0,0,0,1]
])

t1 = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,40],
    [0,0,0,1]
])



### Warning: Experimental Code for DH
Never tested and probably not working

In [ ]:
#warning: rad
#Ansatz DH, nicht überprüft

a1 = rad(90)
a2 = rad(90)
a3 = rad(180)
a4 = rad(0)
a5 = rad(90)


dh_parameters = np.array ([
    [0, rad(90), 71.5, a1],
    [125, 0, 0, a2+rad(90)],
    [125, 0, 0, a3],
    [0, rad(90), 0, a4 - rad(90)],
    [0, 0, 192, a5]
])

dh_matrices = []

for row in dh_parameters:
    dh_matrices.append(dh_calc(row[0],row[1],row[2],row[3]))
    


dh_matrices[0]@dh_matrices[1]@dh_matrices[2]@dh_matrices[3]@dh_matrices[4]

## Reverse Kinematics

In [ ]:
### target = np.array([x,y,z])
target = np.array([240,50,10])
#target = np.array([125,0,0])

h = 1E-8

def f(a):
    
    #Implement three axis of freedom
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(0) @ Tz(60) @ Tz(132)
    
    #res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    delta = target.reshape(3,1) - res[0:3,3:4]
    #print(f'i calc {delta} = {target.reshape(3,1)} - {res[0:3,3:4]}')
    return delta

def f_d(x0):
    y0 = f (x0)
    #print (y0)
    x1 = np.copy (x0); x1 [0] += h; y1 = f (x1); df1 = (y1 - y0) / h;
    x2 = np.copy (x0); x2 [1] += h; y2 = f (x2); df2 = (y2 - y0) / h;
    x3 = np.copy (x0); x3 [2] += h; y3 = f (x3); df3 = (y3 - y0) / h;
    J = np.concatenate ((df1, df2, df3), axis = 1)
    dx = np.linalg.solve (J, y0)
    print (dx)

def rev_k(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    J = np.concatenate((col1_df,col2_df,col3_df),axis=1)
    
    dx = np.linalg.solve (J,y0)
    print(dx)
    print(x0)


#Jacobimatrix
def j(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    return np.concatenate((col1_df,col2_df,col3_df),axis=1)

#Iterative newton-raphson approximation for angle correction
#Input: phi1,phi2,phi3 (Winkel!)
def approximate(x0):
    x = np.copy(x0).reshape(3,1)
    for i in range(15):
        #x -= np.linalg.solve(j(x.reshape(3,)),f(x.reshape(3,)))
        x -= np.linalg.inv(j(x.reshape(3,))) @ f(x.reshape(3,))
        print(f'n={i+1}\n{x}\n')

#approximate(np.array([10,10,20]).astype(float))
f(np.array([10,10,20]).astype(float))
#rev_k(np.array([20,10,20]).astype(float))

In [ ]:
s.close()